In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import math
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from keras.activations import sigmoid
from keras.models import Model ,load_model
from keras.layers import Input, Dense, ConvLSTM2D, Conv2D, Conv1D, MaxPooling2D, Layer, GlobalAveragePooling2D, Reshape, Flatten, BatchNormalization, Bidirectional
from keras.regularizers import L2
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers.legacy import Adam
from keras.saving import register_keras_serializable
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import r2_score
from scipy.sparse.linalg import cg


In [14]:
print(tf.__version__)
print(tf.config.list_physical_devices())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2.14.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPU


In [15]:
# dirs
DATA_DIR = "./load.csv"
TEST_PLOT_DIR = "./result/proposed/"

In [16]:
load_col = 'out.site_energy.total.energy_consumption.kwh'
# MWh

In [17]:
data = pd.read_csv(DATA_DIR)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data[load_col] = data[load_col] * 4 / 1e3


In [18]:
# scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[load_col].to_numpy().reshape(-1, 1))
data[load_col] = data_scaled

joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [7]:
SEED = 42
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

In [8]:
data[load_col]

0        0.307914
1        0.313486
2        0.322878
3        0.332147
4        0.318326
           ...   
35035    0.323867
35036    0.324849
35037    0.310569
35038    0.306187
35039    0.306259
Name: out.site_energy.total.energy_consumption.kwh, Length: 35040, dtype: float64

In [9]:
data.describe()

,upgrade,timestamp,models_used,floor_area_represented,out.district_cooling.cooling.energy_consumption.kwh,out.district_heating.heating.energy_consumption.kwh,out.district_heating.water_systems.energy_consumption.kwh,out.electricity.cooling.energy_consumption.kwh,out.electricity.exterior_lighting.energy_consumption.kwh,out.electricity.fans.energy_consumption.kwh,...,out.electricity.total.energy_consumption.kwh.savings,out.natural_gas.total.energy_consumption.kwh.savings,out.district_heating.cooling.energy_consumption.kwh.savings,out.natural_gas.cooling.energy_consumption.kwh.savings,out.other_fuel.cooling.energy_consumption.kwh.savings,out.other_fuel.heating.energy_consumption.kwh.savings,out.other_fuel.interior_equipment.energy_consumption.kwh.savings,out.other_fuel.total.energy_consumption.kwh.savings,out.other_fuel.water_systems.energy_consumption.kwh.savings,out.site_energy.total.energy_consumption.kwh.savings
count,35040.0,35040,35040.0,3.504000e+04,35040.000000,35040.0,35040.0,35040.000000,35040.000000,35040.000000,...,35040.000000,35040.000000,35040.0,35040.0,35040.0,35040.0,35040.0,35040.0,35040.0,35040.000000
mean,18.0,2018-07-02 12:07:30,535.0,2.933235e+08,351.077590,0.0,0.0,10702.802560,3606.591142,20721.475338,...,4979.811962,23848.451624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28859.420341
min,18.0,2018-01-01 00:15:00,535.0,2.933235e+08,0.000000,0.0,0.0,326.378723,0.000000,16646.463233,...,-28508.867274,262.091277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3108.391514
25%,18.0,2018-04-02 06:11:15,535.0,2.933235e+08,56.488275,0.0,0.0,2706.386049,0.000000,19725.608356,...,-529.918116,8719.805585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17223.696432
50%,18.0,2018-07-02 12:07:30,535.0,2.933235e+08,259.402965,0.0,0.0,7965.151518,3546.633502,20458.531255,...,2273.919309,18191.504378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23244.709423
75%,18.0,2018-10-01 18:03:45,535.0,2.933235e+08,574.228447,0.0,0.0,15479.805245,7127.583771,21985.597167,...,9286.504567,32494.864830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35158.530405
max,18.0,2019-01-01 00:00:00,535.0,2.933235e+08,1559.617664,0.0,0.0,55401.017935,7361.098134,24359.717506,...,40740.822629,159204.018407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136399.915739
std,0.0,NaN,0.0,5.960550e-08,331.710202,0.0,0.0,9898.426784,3564.157927,1548.734876,...,8136.228348,20711.247693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17025.161426


In [10]:
"""
!! parameter settings
n_predict: predict steps
height: final height of the image:
            height * 2 if the n_predict <= width,
            height * 2 + 1 if the n_predict > width
width: width of the image
n_days: use past n days historical time series data as input (number of channel)
n_window_shift: the shift interval of sliding window
"""
n_predict = 2
height = 8
width = 12
n_days = 3
n_window_shift = "15min"

In [11]:
class TimeSeriesImageCoder():
    def __init__(
            self,
        X: pd.DataFrame,
        n_predict: int,
        height: int,
        width: int,
        n_days: int,
        n_window_shift: str
    ) -> None:
        self.X = X
        self.h = height
        self.m = width
        self.d_b = n_days
        self.shift = n_window_shift
        self.n_predict = n_predict
        self.Lb = self.h * self.m
        self.Ls = math.ceil(self.n_predict / self.m) * self.m
        self.timestamps = self.generate_timestamps()
        print(f"Lb: {self.Lb}")
        print(f"Ls: {self.Ls}")

    def generate_timestamps(self):
        start = self.X['timestamp'].min() + DateOffset(days=3)
        end = self.X['timestamp'].max() - DateOffset(minutes=96*15)
        timestamps = pd.date_range(start=start, end=end, freq=self.shift)
        return timestamps
    
    def __make_it_symmetric_3d(self, sets_3d):
        symmetry_training_sets = []
        for slice_2d in np.array(sets_3d):
            reversed_slice_2d = slice_2d[::-1]
            combined_slice_2d = np.concatenate((slice_2d, reversed_slice_2d), axis=0)
            symmetry_training_sets.append(combined_slice_2d)
        return np.array(symmetry_training_sets)
    
    def __make_it_symmetric_2d(self, sets_2d):
        reversed_slice_2d = sets_2d[::-1]
        combined_slice_2d = np.concatenate((sets_2d, reversed_slice_2d), axis=0)
        return np.array(combined_slice_2d)
    

    def encode_b(self):
        training_sets = []
        target_sets = []
        self.X_timeseries_flatten = []
        self.X_timestamp = []
        self.y_timestamp = []
        for steps in self.timestamps:
            training_start_b = steps - DateOffset(days=self.d_b-1, hours=23, minutes=45)
            training_end = steps
            target_start = training_end + DateOffset(minutes=15)
            target_end = steps + DateOffset(minutes=(self.n_predict)*15)
            training_data = self.X[(self.X['timestamp'] >= training_start_b) & (self.X['timestamp'] <= training_end)]
            target_data = self.X[(self.X['timestamp'] >= target_start) & (self.X['timestamp'] <= target_end)]
            if not training_data.empty and not target_data.empty:
                self.X_timeseries_flatten.append(training_data[load_col])
                self.X_timestamp.append(training_data['timestamp'])
                self.y_timestamp.append(target_data['timestamp'])
                training_reshaped = np.array(training_data[load_col]).reshape(self.d_b, self.h, self.m)
                # symmetric_3d = self.__make_it_symmetric_3d(training_reshaped)
                training_sets.append(training_reshaped)
                target_reshaped = np.array(target_data[load_col]).reshape(math.ceil(self.n_predict/self.m), min(self.n_predict, self.m))
                # symmetric_2d = self.__make_it_symmetric_2d(target_reshaped)
                target_sets.append(target_reshaped.flatten())
        training_sets = np.array(training_sets)
        target_sets = np.array(target_sets)

        self.X_timeseries_flatten = np.array(self.X_timeseries_flatten)
        self.X_timestamp = np.array(self.X_timestamp)
        self.y_timestamp = np.array(self.y_timestamp)
        return training_sets, target_sets
    
    def encode(self):
        training_sets_b, target_sets = self.encode_b()
        # training_sets_s = self.encode_s()
        training_sets_b = np.transpose(training_sets_b, (0, 2, 3, 1))
        # training_sets_s = np.transpose(training_sets_s, (0, 2, 3, 1))
        return training_sets_b, target_sets
    
    """calculate the final output of model prediction"""
    def __sum_np(self, matrix):
        n_pairs = len(matrix) // 2
        sums = []
        for i in range(n_pairs):
            sums.append(list(map(sum, zip(matrix[i], matrix[-(i + 1)]))))
        if len(matrix) % 2 != 0:
            sums.append(matrix[n_pairs])

        return [num for row in sums for num in row]
    
    def __x_timeseries_to_image(self, vector):
        matrix_1d = vector.reshape(self.d_b, self.h, self.m)
        image = self.__make_it_symmetric_3d(matrix_1d)
        return image
    
    def pairwise_sum(self, matrix):
        summed_3d_np = np.array([self.__sum_np(layer) for layer in matrix]) / 2
        return summed_3d_np
    
    """Use predictions from previous steps to add new inputs to rolling predictions"""
    def image_shift(self, original_input, new_input):
        input = np.transpose(original_input, (0, 3, 1, 2))
        output = []
        output.append(self.pairwise_sum(input[0]))
        output = np.array(output).flatten()
        output = np.concatenate([output, new_input], axis=0)[-len(output):]
        image = self.__x_timeseries_to_image(output)
        image = image.reshape(1, *image.shape)
        image = np.transpose(image, (0, 2, 3, 1))
        return image


In [12]:
encoder = TimeSeriesImageCoder(
    X=data,
    n_predict=n_predict,
    height=height,
    width=width,
    n_days=n_days,
    n_window_shift=n_window_shift
)
encoded_Xb, encoded_y = encoder.encode()
X_timeseries = np.copy(encoder.X_timeseries_flatten)
X_timestamp = np.copy(encoder.X_timestamp)
y_timestamp = np.copy(encoder.y_timestamp)

Lb: 96
Ls: 12


In [13]:
print(encoded_Xb.shape)
print(encoded_y.shape)

print(X_timeseries.shape)
print(X_timestamp.shape)
print(y_timestamp.shape)

(34656, 8, 12, 3)
(34656, 2)
(34656, 288)
(34656, 288)
(34656, 2)


In [14]:
MONTH_TIME_STEP = math.floor(encoder.timestamps.shape[0] / 24)
X_test_b = []
y_test = []
X_test_b_flatten = []
X_test_b_timestamp = []
y_test_timestamp = []

for i in range(0, 24):
    start = (i+1)*MONTH_TIME_STEP-(192*(i+1))
    end = (i+1)*MONTH_TIME_STEP-(192*i)
    X_test_b.append(encoded_Xb[start:end])
    y_test.append(encoded_y[start:end])
    X_test_b_flatten.append(X_timeseries[start:end])
    X_test_b_timestamp.append(X_timestamp[start:end])
    y_test_timestamp.append(y_timestamp[start:end])


    encoded_Xb = np.concatenate([encoded_Xb[:start], encoded_Xb[end:]])
    encoded_y = np.concatenate([encoded_y[:start], encoded_y[end:]])
    X_timeseries = np.concatenate([X_timeseries[:start], X_timeseries[end:]])
    X_timestamp = np.concatenate([X_timestamp[:start], X_timestamp[end:]])
    y_timestamp = np.concatenate([y_timestamp[:start], y_timestamp[end:]])


In [15]:
 
X_test_b = np.concatenate([i for i in X_test_b])
y_test = np.concatenate([i for i in y_test])
X_test_b_flatten = np.concatenate([i for i in X_test_b_flatten])
X_test_b_timestamp = np.concatenate([i for i in X_test_b_timestamp])
y_test_timestamp = np.concatenate([i for i in y_test_timestamp])




In [16]:
X_train_b = encoded_Xb
y_train = encoded_y

In [17]:
print(np.array(X_train_b).shape)
print(np.array(X_test_b).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)
print(X_test_b_flatten.shape)
print(X_test_b_timestamp.shape)
print(y_test_timestamp.shape)

(30048, 8, 12, 3)
(4608, 8, 12, 3)
(30048, 2)
(4608, 2)
(4608, 288)
(4608, 288)
(4608, 2)


In [18]:
@register_keras_serializable('ECALayer')
class ECALayer(Layer):
    def __init__(self, gamma=2, b=1, **kwargs):
        super(ECALayer, self).__init__(**kwargs)
        self.gamma = gamma
        self.b = b

    def build(self, input_shape):
        c = input_shape[-1]
        self.t = max(1, int(abs((tf.math.log(float(c)) / tf.math.log(2.0) + self.b) / self.gamma)))
        self.conv = Conv1D(filters=1, kernel_size=self.t, padding='same', use_bias=False)
        super(ECALayer, self).build(input_shape)

    def call(self, inputs):
        # Global Average Pooling over the spatial dimensions to produce a (batch_size, 1, channels) tensor
        x = GlobalAveragePooling2D()(inputs)
        x = Reshape((1, -1))(x)
        x = self.conv(x)
        x = sigmoid(x)
        x = tf.squeeze(x, axis=1)  # Squeeze to make it (batch_size, channels)
        
        # Multiply weights across channels
        return inputs * x[:, tf.newaxis, tf.newaxis, :]

    def get_config(self):
        config = super(ECALayer, self).get_config()
        config.update({
            'gamma': self.gamma,
            'b': self.b
        })
        return config

In [19]:
def create_model(input_shape_b, encoder):
    height, width = math.ceil(encoder.n_predict / encoder.m), min(encoder.n_predict, encoder.m)
    inputs_b = Input(shape=input_shape_b)
    conv1 = Conv2D(filters=32, kernel_size=3, padding="same", activation="tanh")(inputs_b)
    conv2 = Conv2D(filters=64, kernel_size=3, padding="same", activation="tanh")(conv1)
    conv2 = Reshape((1, *conv2.shape[1:]))(conv2)
    nor1 = BatchNormalization()(conv2)
    lstm1 = Bidirectional(ConvLSTM2D(filters=96, kernel_size=3, padding="same", activation="tanh", return_sequences=True, dropout=0.0))(nor1)
    nor2 = BatchNormalization()(lstm1)
    lstm2 = Bidirectional(ConvLSTM2D(filters=96, kernel_size=3, padding="same", activation="tanh", return_sequences=False, dropout=0.0))(nor2)
    nor3 = BatchNormalization()(lstm2)
    eca1 = ECALayer()(nor3)
    nor4 = BatchNormalization()(eca1)
    conv3 = Conv2D(filters=64, kernel_size=3, padding="same", activation="tanh")(nor4)
    conv4 = Conv2D(filters=32, kernel_size=3, padding="same", activation="tanh")(conv3)
    nor5 = BatchNormalization()(conv4)
    maxpool1 = MaxPooling2D(pool_size=10, padding="same")(nor5)
    flatten1 = Flatten()(maxpool1)
    outputs = Dense(height*width, activation="linear")(flatten1)
    print(height*width)
    model = Model(inputs=inputs_b, outputs=outputs)

    return model

In [20]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

%load_ext tensorboard

tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor="loss", patience=10, min_delta=5e-5)
reduce_lr_callback = ReduceLROnPlateau(monitor="loss", factor=0.3, patience=5, verbose=1, min_lr=1e-7)
callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback]
model = create_model(input_shape_b=X_train_b.shape[1:], encoder=encoder)
model.compile(optimizer=Adam(learning_rate=5e-5), loss="mse")
model.summary()

2
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 12, 3)]        0         
                                                                 
 conv2d (Conv2D)             (None, 8, 12, 32)         896       
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 12, 64)         18496     
                                                                 
 reshape (Reshape)           (None, 1, 8, 12, 64)      0         
                                                                 
 batch_normalization (Batch  (None, 1, 8, 12, 64)      256       
 Normalization)                                                  
                                                                 
 bidirectional (Bidirection  (None, 1, 8, 12, 192)     1106688   
 al)                                                       

In [21]:
history = model.fit(
    X_train_b,
    y_train,
    verbose=1,
    epochs=120,
    batch_size=96,
    callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback]
)

Epoch 1/120


2024-06-14 17:17:32.224169: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'Func/gradient_tape/model/bidirectional_1/backward_conv_lstm2d_1/while/model/bidirectional_1/backward_conv_lstm2d_1/while_grad/body/_948/input/_2185' -> 'gradient_tape/model/bidirectional_1/backward_conv_lstm2d_1/while/model/bidirectional_1/backward_conv_lstm2d_1/while_grad/body/_948/gradient_tape/model/bidirectional_1/backward_conv_lstm2d_1/while/gradients/AddN', 'Func/gradient_tape/model/bidirectional_1/forward_conv_lstm2d_1/while/model/bidirectional_1/forward_conv_lstm2d_1/while_grad/body/_753/input/_2066' -> 'gradient_tape/model/bidirectional_1/forward_conv_lstm2d_1/while/model/bidirectional_1/forward_conv_lstm2d_1/while_grad/body/_753/gradient_tape/model/bidirectional_1/forward_conv_lstm2d_1/while/gradients/AddN', 'Func/gradient_tape/model/bidirectional/backward_conv_lstm2d/while/mod

313/313 [==============================] - 18s 47ms/step - loss: 0.0837 - lr: 5.0000e-05
Epoch 2/120
313/313 [==============================] - 15s 47ms/step - loss: 0.0036 - lr: 5.0000e-05
Epoch 3/120
313/313 [==============================] - 15s 47ms/step - loss: 0.0024 - lr: 5.0000e-05
Epoch 4/120
313/313 [==============================] - 15s 47ms/step - loss: 0.0017 - lr: 5.0000e-05
Epoch 5/120
313/313 [==============================] - 15s 47ms/step - loss: 0.0014 - lr: 5.0000e-05
Epoch 6/120
313/313 [==============================] - 15s 48ms/step - loss: 0.0012 - lr: 5.0000e-05
Epoch 7/120
313/313 [==============================] - 15s 48ms/step - loss: 0.0011 - lr: 5.0000e-05
Epoch 8/120
313/313 [==============================] - 15s 48ms/step - loss: 9.6502e-04 - lr: 5.0000e-05
Epoch 9/120
313/313 [==============================] - 15s 48ms/step - loss: 9.0948e-04 - lr: 5.0000e-05
Epoch 10/120
313/313 [==============================] - 15s 48ms/step - loss: 9.6568e-04 - lr: 

In [22]:
y_pred = model.predict([X_test_b])

  9/144 [>.............................] - ETA: 0s 

2024-06-14 17:28:06.700574: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'model/bidirectional_1/forward_conv_lstm2d_1/while/body/_97/model/bidirectional_1/forward_conv_lstm2d_1/while/Tanh_1' -> 'model/bidirectional_1/forward_conv_lstm2d_1/while/body/_97/model/bidirectional_1/forward_conv_lstm2d_1/while/mul_5', 'Func/model/bidirectional_1/forward_conv_lstm2d_1/while/body/_97/input/_288' -> 'model/bidirectional_1/forward_conv_lstm2d_1/while/body/_97/model/bidirectional_1/forward_conv_lstm2d_1/while/mul_2', 'model/bidirectional_1/forward_conv_lstm2d_1/while/body/_97/model/bidirectional_1/forward_conv_lstm2d_1/while/convolution_7' -> 'model/bidirectional_1/forward_conv_lstm2d_1/while/body/_97/model/bidirectional_1/forward_conv_lstm2d_1/while/add_6', 'model/bidirectional_1/backward_conv_lstm2d_1/while/body/_145/model/bidirectional_1/backward_conv_lstm2d_1/while/Tan

144/144 [==============================] - 1s 7ms/step


In [23]:
print(y_test.shape)
print(y_pred.shape)

mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("-" * 86)
print(f'mse: {mse:.4f}')
print(f'rmse: {rmse:.4f}')
print(f'mae: {mae:.4f}')
print(f'mape: {mape: .4f}')
print(f'r2: {r2:.4f}')
print("-" * 86)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)


mse_inv = mean_squared_error(y_test_inv, y_pred_inv)
rmse_inv = math.sqrt(mse_inv)
mae_inv = mean_absolute_error(y_test_inv, y_pred_inv)
mape_inv = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
r2_inv = r2_score(y_test_inv, y_pred_inv)

print("-" * 86)
print(f'mse_inv: {mse_inv:.4f}')
print(f'rmse_inv: {rmse_inv:.4f}')
print(f'mae_inv: {mae_inv:.4f}')
print(f'mape_inv: {mape_inv: .4f}')
print(f'r2_inv: {r2_inv:.4f}')
print("-" * 86)

(4608, 2)
(4608, 2)
--------------------------------------------------------------------------------------
mse: 0.0002
rmse: 0.0135
mae: 0.0098
mape:  0.0404
r2: 0.9939
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
mse_inv: 36.9168
rmse_inv: 6.0759
mae_inv: 4.4276
mape_inv:  0.0115
r2_inv: 0.9939
--------------------------------------------------------------------------------------


In [24]:
def calculate_metrics_per_steps(true_values, predicted_values):
    n_steps = true_values.shape[1]

    mse = []
    rmse = []
    mae = []
    mape = []

    for i in range(n_steps):
        true_step = true_values[:, i]
        predicted_step = predicted_values[:, i]

        mse_step = mean_squared_error(true_step, predicted_step)
        rmse_step = np.sqrt(mse_step)
        mae_step = mean_absolute_error(true_step, predicted_step)
        mape_step = mean_absolute_percentage_error(true_step, predicted_step)

        mse.append(mse_step)
        rmse.append(rmse_step)
        mae.append(mae_step)
        mape.append(mape_step)

    return np.array(mse), np.array(rmse), np.array(mae), np.array(mape)

In [25]:
mse_per_steps, rmse_per_steps, mae_per_steps, mape_per_steps = calculate_metrics_per_steps(y_test_inv, y_pred_inv)

In [26]:
print(mse_per_steps)
print(rmse_per_steps)
print(mae_per_steps)
print(mape_per_steps)

[35.4091175 38.4243862]
[5.95055607 6.19874069]
[4.33763287 4.51759689]
[0.01133938 0.01163342]


In [27]:
model.save("./model/proposed_2steps.keras")